# Watershed Distance Transform for 2D Data
---
Implementation of papers:

[Deep Watershed Transform for Instance Segmentation](http://openaccess.thecvf.com/content_cvpr_2017/papers/Bai_Deep_Watershed_Transform_CVPR_2017_paper.pdf)

[Learn to segment single cells with deep distance estimator and deep cell detector](https://arxiv.org/abs/1803.10829)

In [1]:
import os
import errno
import numpy as np
import deepcell
from tensorflow.python import keras


/usr/local/lib/python3.5/dist-packages/deepcell/utils/__init__.py:49: UserWarning: To use `compute_overlap`, the C extensions must be built using `python setup.py build_ext --inplace`
  warnings.warn('To use `compute_overlap`, the C extensions must be built '


In [2]:
from tensorflow.keras.optimizers import SGD
from deepcell.utils.train_utils import rate_scheduler

test_size = .10  # % of data saved as test
norm_method = 'std'  # data normalization
receptive_field = 81  # should be adjusted for the scale of the data

optimizer = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

lr_sched = rate_scheduler(lr=0.01, decay=0.99)

# Sample mode settings
batch_size = 64  # number of images per batch (should be 2 ^ n)
win = (receptive_field - 1) // 2  # sample window size
balance_classes = True  # sample each class equally
max_class_samples = 1e6  # max number of samples per class

# Transformation settings
transform = 'watershed'
distance_bins = 4  # number of distance "classes"
erosion_width = 3  # erode edges
LOG_DIR = '/data/logs'

In [3]:
base_name = 'Training_Freeze_1_Nuc'
npz_name = '/data/npz_data/' + base_name + ".npz"
MODEL_DIR = '/data/models/' + '20190822_training_freeze_1'
sample_model_name = base_name + "_watershed_81_rf_256_dense_64_conv_3erosion"

if not os.path.isdir(MODEL_DIR):
    os.makedirs(MODEL_DIR)

### Load the Training Data

In [4]:
CHANNEL_AXIS = 3
training_data = np.load(npz_name)

X, y = training_data["X"], training_data["y"]
print("X.shape: {} & y.shape: {}".format(X.shape, y.shape))

X.shape: (4, 1024, 1024, 5) & y.shape: (4, 1024, 1024, 1)


### Create a model for the watershed energy transform

#### Instantiate the distance transform model

In [5]:
from deepcell import model_zoo
from deepcell.training import train_model_sample

watershed_model = model_zoo.bn_feature_net_2D(
    receptive_field=receptive_field,
    n_channels=X.shape[CHANNEL_AXIS],
    n_features=distance_bins,
    n_conv_filters=64,
    n_dense_filters=256)

In [6]:
sample_model_name

'Training_Freeze_1_Nuc_watershed_81_rf_256_dense_64_conv_3erosion'

In [8]:
watershed_model.load_weights("/data/models/20190822_training_freeze_1/Training_Freeze_1_Nuc_watershed_81_rf_256_dense_64_conv_2erosion_epoch_12.h5")

#### Train the model

In [ ]:
watershed_model = train_model_sample(
    model=watershed_model,
    dataset=npz_name, 
    model_name=sample_model_name,
    test_size=test_size,
    optimizer=optimizer,
    batch_size=batch_size,
    n_epoch=6,
    window_size=(win, win),
    transform=transform,
    distance_bins=distance_bins,
    erosion_width=erosion_width,
    balance_classes=True,
    max_class_samples=240000,
    model_dir=MODEL_DIR,
    log_dir=LOG_DIR,
    lr_sched=lr_sched,
    rotation_range=180,
    flip=True,
    shear=False,
    zoom_range=(0.8, 1.2),
    val_monitor=False,
    save_period=1)

using all data as training data
Using class weights of {0: 1.0, 1: 1.0, 2: 1.0, 3: 1.0}
X_train shape: (4, 1024, 1024, 5)
y_train shape: (4, 1024, 1024, 1)
Output Shape: (None, 4)
Number of Classes: 4
Training on 1 GPUs
the max_class_samples per image is 60000
analyzing image 0
the least represented class has 34576 examples
analyzing class 0
downsampling from 664627 examples per class
analyzing class 1
downsampling from 114784 examples per class
analyzing class 2
downsampling from 75262 examples per class
analyzing class 3
downsampling from 34576 examples per class
analyzing image 1
the least represented class has 32630 examples
analyzing class 0
downsampling from 683976 examples per class
analyzing class 1
downsampling from 103169 examples per class
analyzing class 2
downsampling from 69474 examples per class
analyzing class 3
downsampling from 32630 examples per class
analyzing image 2
the least represented class has 26537 examples
analyzing class 0
downsampling from 718049 examples 